Salency maps para NLP utilizando AllenNLP
=========================================

Introducción
------------

AllenNLP es un framework general de aprendizaje profundo para NLP, establecido por el mundialmente famoso Allen Institute for AI Lab. Contiene modelos de referencia de última generación que se ejecutan sobre el `PyTorch`. AllenNLP es una librería que ademas busca implementar abstracciones que permitan el rápido desarrollo de modelos y reutilización de componentes al despegarse de detalles de implementación de cada modelo.

En este ejemplo, veremos como utilizar esta librería para generar salency maps utilizando los gradientes de las prediciones. Esto nos permita interpretar las predicciones de nuestros modelos basados en `transformers`.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/allennlp_interpret.txt \
    --quiet --no-clobber
!pip install -r allennlp_interpret.txt --quiet

Si ejecuta en Google Colab, adicionalmente deberá:

In [ ]:
!pip install -U google-cloud-storage==1.40.0

Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [3]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip -qq tweet_classification_bert.zip

Cargamos el set de datos

In [1]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Cargando un modelo entreando con Transformers en AllenNLP
---------------------------------------------------------

`allennlp` es un framework compatible con la libraría `transformers` lo cual resulta atractivo a la hora de utilizar modelos que son entrenados en una para luego llevarlo a la otra. Veamos entonces como podemos hacer para cargar el modelo que tenemos previamente entrenado para la clasificación de tweets utilizando una arquitectura `BERT` dentro de este framework. En particular, nuestro modelo se persistió en el directorio "tweet_classification".

In [2]:
model_name = "tweet_classification_bert"

### Creando un objeto Model

Importamos algunos elementos que necesitaremos

In [3]:
from typing import Dict, Iterable, List

from allennlp.common import Params
from allennlp.data import DatasetReader, Instance, Batch
from allennlp.data.fields import Field, LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Tokenizer
from allennlp.data.vocabulary import PreTrainedTokenizer, Vocabulary
from allennlp.models import BasicClassifier, Model
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.data.tokenizers.pretrained_transformer_tokenizer import PretrainedTransformerTokenizer
from allennlp.data.token_indexers.pretrained_transformer_indexer import PretrainedTransformerIndexer
from allennlp.modules.seq2vec_encoders.bert_pooler import BertPooler

Cargaremos todos los elementos que son necesarios para utilizar esta libreria. Todos ellos son generados a partir del modelo que persistimos en `transformers`. La utilidad de cada uno de estos módulos esta fuera del alcance de este curso pero recomendamos revisar la documentación de AllenNLP para más información sobre cual es su rol.

In [4]:
transformer_vocab = Vocabulary.from_pretrained_transformer(model_name)
transformer_tokenizer = PretrainedTransformerTokenizer(model_name)
transformer_encoder = BertPooler(model_name)

token_indexer = PretrainedTransformerIndexer(model_name)

Some weights of the model checkpoint at tweet_classification_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


> ¿Notan el mensaje de advertencia? Lo correjiremos mas adelante.

In [5]:
params = Params(
    {
     "token_embedders": {
        "tokens": {
          "type": "pretrained_transformer",
          "model_name": model_name,
        }
      }
    }
)

token_embedder = BasicTextFieldEmbedder.from_params(vocab=transformer_vocab, params=params)

Creamos el modelo a partir de todos los componentes que cargamos anteriormente:

In [6]:
model = BasicClassifier(vocab=transformer_vocab, text_field_embedder=token_embedder, seq2vec_encoder=transformer_encoder, dropout=0.1, num_labels=7)

A continuación, cargaremos los pesos del clasificador.

In [7]:
from transformers import BertForSequenceClassification
classifier = BertForSequenceClassification.from_pretrained(model_name)

In [8]:
model._classification_layer.weight = classifier.classifier.weight
model._classification_layer.bias = classifier.classifier.bias

Configuremos el modelo para trabajar en modo `inferencia`:

In [9]:
_ = model.eval()

### Creamos un DatasetReader

AllenNLP utiliza un objeto llamado `DatasetReader` que le permite crear `Instance`'s de datos que son suministradas al modelo. Esta abstracción permite realizar cualquier preprocesamiento que es necesario antes de enviar los datos al modelo. Debemos generar nuestro propia implementación para el caso de clasificación utilizando un modelo basado en *transformers*. La siguiente clase realiza esto: 

In [10]:
from allennlp.data import DatasetReader

In [11]:
class ClassificationTransformerReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexer: TokenIndexer,
        max_tokens: int,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer
        self.token_indexers: Dict[str, TokenIndexer] = { "tokens": token_indexer }
        self.max_tokens = max_tokens

    def text_to_instance(self, text: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(text)
        if self.max_tokens:
            tokens = tokens[: self.max_tokens]
        
        inputs = TextField(tokens, self.token_indexers)
        fields: Dict[str, Field] = { "tokens": inputs }
            
        if label:
            fields["label"] = LabelField(label)
            
        return Instance(fields)

Instanciamos el `DatasetReader`

In [12]:
dataset_reader = ClassificationTransformerReader(tokenizer=transformer_tokenizer, token_indexer=token_indexer, max_tokens=400)

### Provemos que nuestro modelo funciona

Unamos todas las piezas que generamos hasta el momento probandolo sobre una instancia:

In [13]:
sample_text = "Movistar no tiene señal ni en su casa"
instance = dataset_reader.text_to_instance(sample_text)

In [14]:
from allennlp.nn import util

dataset = Batch([instance])
dataset.index_instances(transformer_vocab)
model_input = util.move_to_device(dataset.as_tensor_dict(), model._get_prediction_device())

In [15]:
outputs = model.make_output_human_readable(model(**model_input))
print(outputs['probs'].argmax())

tensor(6)


Recordemos que en el conjunto de datos de entrenamiento, las etiquetas se distribuyen como sigue:

```
{
    'ALIMENTACION': 0,
    'AUTOMOCION': 1,
    'BANCA': 2,
    'BEBIDAS': 3,
    'DEPORTES': 4,
    'RETAIL': 5,
    'TELCO': 6
}
```

Interpretando nuestras predicciones
-----------------------------------

Una vez que tenemos nuestro modelo correctamente cargado, veamos como podemos interpretar una predicción computando el salency map a partir de los gradientes.

In [16]:
from allennlp.interpret.saliency_interpreters import SimpleGradient
from allennlp.predictors import Predictor, TextClassifierPredictor

predictor = TextClassifierPredictor(model, dataset_reader)
interpreter = SimpleGradient(predictor)

Busquemos un tweet para interpretar:

In [36]:
sample_text_idx = 1522
sample_text = tweets['TEXTO'][sample_text_idx]
sample_label = tweets['SECTOR'][sample_text_idx]

print("Texto:", sample_text, "\Sector:", sample_label)

Texto: @HyundaiPeru con Grupo Primax realiza este verano servicios de Inspección Digital Gratuita a vehículos Hyundai en e… https://t.co/TZ4XFziOd3 \Sector: AUTOMOCION


Calculemos los gradientes para cada token:

In [37]:
inputs = {"sentence": sample_text }

In [38]:
interpretation = interpreter.saliency_interpret_from_json(inputs)
grads = interpretation['instance_1']['grad_input_1']

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Podemos graficar los resultados utilizando un mapa de calor marcando con colores más intensos aquellos tokens que tienen mayor impacto en las predicciones:

In [39]:
import math
from IPython.display import HTML

html = ""
for idx, token in enumerate(transformer_tokenizer.tokenize(inputs['sentence'])):
    html += "<span style='background-color:rgba(255,0,0,{})'>{} </span>".format(grads[idx],token)
    
HTML(html)